In [79]:
import pandas as pd
import numpy as np
import sqlite3
import re

conn = sqlite3.connect("profiles.db")
cursor = conn.cursor()
cursor.execute("ATTACH DATABASE 'colleges.db' AS colleges")

In [119]:
#TODO: Cluster Universities + Get Mean Values + Reassign to profiles on join
import re, math
from sklearn.cluster import KMeans

def get_type_map(df_column):
    return {t:i for i, t in enumerate(set(df_column))}

df = pd.read_sql_query("SELECT * from colleges.university_profiles where institution_type like '%Public%'",conn)
dfc = df.drop("school", axis=1)
# CLEAN UNIVERSITY DATA
# drop sat averages
for col in ['sat_math', 'sat_reading', 'sat_writing', 'average_freshman_aid', 'international_percent', 'city', 'state', 'institution_type']:
    dfc = dfc.drop(col, axis=1)
    df = df.drop(col,axis=1)

# remove duplicate school column
dfc = dfc.loc[:,~dfc.columns.duplicated()]
df = df.loc[:,~df.columns.duplicated()]

# handle nones in all columns
dfc.fillna(value=pd.np.nan, inplace=True)
df.fillna(value=pd.np.nan, inplace=True)
# remove - in avg_act
dfc['avg_act'] = dfc['avg_act'].map(lambda x: str(x).strip('-'))
df['avg_act'] = df['avg_act'].map(lambda x: str(x).strip('-'))
# cost attendance in state vs out state
dfc['cost_attendance'] = dfc['cost_attendance'].map(lambda x: x[x.find("$")+1:x.find("<")-1] if not isinstance(x,float) else x)
df['cost_attendance'] = df['cost_attendance'].map(lambda x: x[x.find("$")+1:x.find("<")-1] if not isinstance(x,float) else x)
# fix faculty total none as string bug
dfc["faculty_total"] = dfc["faculty_total"].map(lambda x : None if x == "None" else x)
df["faculty_total"] = df["faculty_total"].map(lambda x : None if x == "None" else x)
# weird act nan bug
dfc = dfc[dfc.avg_act != "nan"]
df = df[df.avg_act != "nan"]

dfc = dfc[dfc.avg_act != None]
df = df[df.avg_act != None]

dfc = dfc[dfc.avg_gpa != None]
df = df[df.avg_gpa != None]

dfc = dfc.dropna()
df = df.dropna()

dfc = dfc[dfc.avg_act != '9 a']
df = df[df.avg_act != '9 a']

def getNextTableIndex(index,j):
    isValid = False
    while(not isValid):
        try:
            df['avg_gpa'][index]
            return index, j+1
        except:
            index += 1
            
points = []
i = 0
j=0
while i < len(dfc.index):
    i, j = getNextTableIndex(i,j)
    point = []
    for col in dfc.columns.values:
        point.append(dfc[col].get(i))
    points.append(point)
    i += 1
cluster = KMeans(n_clusters=240, max_iter=500).fit(points)
i = 0
j = 0
while i < len(dfc.index):
    label = cluster.labels_[j]
    i, j = getNextTableIndex(i,j)
    for c,col in enumerate(dfc.columns.values):
        df.set_value(i, col, cluster.cluster_centers_[label][c])
    i += 1
dfc = df

print(dfc)

                                                school avg_gpa avg_act  \
0                         Ramapo College of New Jersey    3.37      21   
2              Shippensburg University of Pennsylvania   3.255    21.5   
4                           Penn State University Park    3.58      28   
5             Slippery Rock University of Pennsylvania    3.42      22   
6                 Penn State Erie, The Behrend College    3.31      22   
7                University of Massachusetts Dartmouth    3.23      23   
8                Southern Connecticut State University       3      20   
9                                Penn State Harrisburg    3.22      22   
10                        University of Southern Maine    3.16      19   
11                          University of Rhode Island    3.53      26   
13                                University at Albany     3.2      24   
14                               Binghamton University     3.7      29   
15                               Unive

/home/marclane/.local/lib/python3.6/site-packages/ipykernel_launcher.py:74: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [120]:
df = pd.read_sql_query("SELECT * FROM profiles",conn)

In [121]:
def get_type_map(df_column):
    return {t:i for i, t in enumerate(set(df_column))}

def convert_sat_to_act(sat):
    if np.isnan(sat):
        return np.nan
    sat_scores = np.array([990, 1060, 1140, 1210, 1270, 1330, 1390, 1450, 1510, 1560, 1620, 1680, 1740, 1800, 1860, 1920, 1980, 2020, 2080, 2140, 2220, 2290, 2380, 2410])
    return np.where(sat < sat_scores)[0][0] + 13
    
def combine_test_scores(x):
    if not np.isnan(x['act']):
        return x['act']/36.0
    sat = np.array([x['sat_m'], x['sat_r'], x['sat_w']])
    return convert_sat_to_act(3*sat[~np.isnan(sat)].mean())/36.0 if len(sat[np.isnan(sat)]) != 3 else np.nan

def convert_string(x):
    try:
        return int(x)
    except:
        return np.nan

def combine_avg_test_scores(x):
    if not np.isnan(convert_string(x['avg_act'])):
        return convert_string(x['avg_act'])/36.0
    sat = np.array([convert_string(x['sat_math']), convert_string(x['sat_reading']), convert_string(x['sat_writing'])])
    return convert_sat_to_act(3*sat[~np.isnan(sat)].mean())/36.0 if len(sat[np.isnan(sat)]) != 3 else np.nan

def map_statuses(x):
    status_map = {"Denied": 0, "Deferred": 0, "Wait-Listed": 0, "Accepted": 2, "Will Attend": 2}
    return status_map[x['status']]

def convert_class_rank(x):
    s = str(x['class_rank'])
    if " of " in s:
        return float(s[:s.index(" of ")])/float(s[s.index(" of ") + 4:])
    if "Top " in s:
        return float(s[4:s.index("%")])/100.0
    if "Bottom " in s:
        return float(s[7:s.index("%")])/100.0
    return np.nan

def convert_instate_tuition(x):
    if "<br>" in str(x['cost_attendance']):
        instate = x['cost_attendance'].split("<br>")[0]
        return instate[instate.index("$")+1:]
    else:
        return x['cost_attendance']

def convert_outstate_tuition(x):
    if "<br>" in str(x['cost_attendance']):
        outstate = x['cost_attendance'].split("<br>")[1]
        return outstate[outstate.index("$")+1:]
    else:
        return x['cost_attendance']

def get_tuition(x):
    return x['in_state_tuition'] if x['state'] == x['hs_state'] else x['out_state_tuition']

def map_hs_types(x):
    hs_type_map = {"Public": 0, "Private": 1, "Parochial": 2, "Home": 3}
    return hs_type_map[x['hs_type']]

def map_ins_types(x):
    ins_type_map = {"Public": 0, "Private": 1, "Private for-profit": 1, "&nbsp;": np.nan}
    return ins_type_map[x['institution_type']]

In [122]:
df = df.loc[:,~df.columns.duplicated()]
df.fillna(value=pd.np.nan, inplace=True)
df['test_score'] = df.apply(combine_test_scores, axis=1)
df['class_rank'] = df.apply(convert_class_rank, axis=1)
#df['tuition'] = df.apply(get_tuition, axis=1)
df = pd.merge(dfc, df, left_on = 'school', right_on = 'school')
print(df)
for col in ['school', 'sat_m', 'sat_r', 'sat_w', 'act', 'avg_act', 'gpa_w']:
    df = df.drop(col, axis=1)
df = df.dropna()#subset=['test_score','avg_test_score','gpa_uw','class_rank','hs_state','hs_type','institution_type'])

df['status'] = df.apply(map_statuses, axis=1)
df['hs_type'] = df.apply(map_hs_types, axis=1)
replace_columns = ['hs_state', 'gender']
for col in replace_columns:
    df[col].replace(get_type_map(df[col]), inplace=True)


                                         school avg_gpa avg_act  \
0                  Ramapo College of New Jersey    3.37      21   
1                  Ramapo College of New Jersey    3.37      21   
2                  Ramapo College of New Jersey    3.37      21   
3                  Ramapo College of New Jersey    3.37      21   
4                  Ramapo College of New Jersey    3.37      21   
5                  Ramapo College of New Jersey    3.37      21   
6                  Ramapo College of New Jersey    3.37      21   
7                  Ramapo College of New Jersey    3.37      21   
8                  Ramapo College of New Jersey    3.37      21   
9                  Ramapo College of New Jersey    3.37      21   
10                 Ramapo College of New Jersey    3.37      21   
11                 Ramapo College of New Jersey    3.37      21   
12                 Ramapo College of New Jersey    3.37      21   
13                 Ramapo College of New Jersey    3.37      2

In [123]:
print(len(df.index))

22758


In [124]:
df.sample(n=100)

,avg_gpa,cost_attendance,admission_rate,faculty_total,female_percentage,year,gender,hs_type,hs_state,gpa_uw,class_rank,status,eaed,legacy,athlete,test_score
29284,3.71,2779,65,1198,57.6,2015,1,0,52,3.85,0.073477,2,0,0,0,0.694444
29460,3.56,2336,74,1021,49.2,2017,2,2,43,3.30,0.100000,2,0,1,0,0.944444
14596,3.7,2822,35,876,54.2,2015,2,1,16,3.77,0.074380,2,0,0,0,0.833333
222,3.58,3634,50,2679,46.7,2012,1,0,2,3.20,0.289398,2,0,0,0,0.694444
20851,3.24,2111,100,597,56.5,2012,1,0,4,3.12,0.435685,2,0,0,0,0.583333
21969,3.93,2250,50,1572,54.3,2019,1,0,52,3.80,0.100000,2,0,0,0,0.805556
838,3.58,3634,50,2679,46.7,2018,1,0,52,3.80,0.100000,2,0,0,0,0.777778
31633,3.08,2363,47,419,60.8,2009,1,0,53,2.96,0.500000,2,1,0,0,0.694444
8499,3.86,3128,84,1330,48.7,2013,2,0,9,3.35,0.403259,2,0,0,0,0.583333
3792,3.26,2423,92,512,47.2,2010,2,0,40,3.70,0.100000,2,0,0,0,0.833333


In [125]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.linear_model import LogisticRegression

k = 1
df0 = df[df.status == 0]
df2 = df[df.status == 2].sample(n = int(k*len(df0)))
df_final = pd.concat([df0, df2])
X = df_final.drop('status', axis=1)
y = df_final['status']
X_train, X_test, y_train, y_test = train_test_split(X, y)

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

mlp = MLPClassifier(hidden_layer_sizes=(8,8,8),max_iter=1000)
mlp.fit(X_train,y_train)
predictions = mlp.predict(X_test)
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))

logisticRegr = LogisticRegression()
logisticRegr.fit(X_train, y_train)
score = logisticRegr.score(X_test, y_test)
print("Logistic Regression Score: " + str(score))

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/marclane/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
/home/marclane/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.


[[758 158]
 [201 681]]
              precision    recall  f1-score   support

           0       0.79      0.83      0.81       916
           2       0.81      0.77      0.79       882

   micro avg       0.80      0.80      0.80      1798
   macro avg       0.80      0.80      0.80      1798
weighted avg       0.80      0.80      0.80      1798

Logistic Regression Score: 0.7764182424916574


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
